![tracker](https://us-central1-vertex-ai-mlops-369716.cloudfunctions.net/pixel-tracking?path=statmike%2Fvertex-ai-mlops%2FFramework+Workflows%2FPyTorch%2Fserving&file=dataflow-streaming-runinference.ipynb)
<!--- header table --->
<table align="left">
<tr>
  <td style="text-align: center">
    <a href="https://github.com/statmike/vertex-ai-mlops/blob/main/Framework%20Workflows/PyTorch/serving/dataflow-streaming-runinference.ipynb">
      <img width="32px" src="https://www.svgrepo.com/download/217753/github.svg" alt="GitHub logo">
      <br>View on<br>GitHub
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/statmike/vertex-ai-mlops/blob/main/Framework%20Workflows/PyTorch/serving/dataflow-streaming-runinference.ipynb">
      <img width="32px" src="https://www.gstatic.com/pantheon/images/bigquery/welcome_page/colab-logo.svg" alt="Google Colaboratory logo">
      <br>Run in<br>Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https%3A%2F%2Fraw.githubusercontent.com%2Fstatmike%2Fvertex-ai-mlops%2Fmain%2FFramework%2520Workflows%2FPyTorch%2Fserving%2Fdataflow-streaming-runinference.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo">
      <br>Run in<br>Colab Enterprise
    </a>
  </td>      
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/bigquery/import?url=https://github.com/statmike/vertex-ai-mlops/blob/main/Framework%20Workflows/PyTorch/serving/dataflow-streaming-runinference.ipynb">
      <img width="32px" src="https://www.gstatic.com/images/branding/gcpiconscolors/bigquery/v1/32px.svg" alt="BigQuery logo">
      <br>Open in<br>BigQuery Studio
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/statmike/vertex-ai-mlops/main/Framework%20Workflows/PyTorch/serving/dataflow-streaming-runinference.ipynb">
      <img width="32px" src="https://www.gstatic.com/images/branding/gcpiconscolors/vertexai/v1/32px.svg" alt="Vertex AI logo">
      <br>Open in<br>Vertex AI Workbench
    </a>
  </td>
</tr>
<tr>
  <td colspan="5" style="text-align: right">
    <b>Share This On: </b> 
    <a href="https://www.linkedin.com/sharing/share-offsite/?url=https://github.com/statmike/vertex-ai-mlops/blob/main/Framework%2520Workflows/PyTorch/serving/dataflow-streaming-runinference.ipynb"><img src="https://upload.wikimedia.org/wikipedia/commons/8/81/LinkedIn_icon.svg" alt="Linkedin Logo" width="20px"></a> 
    <a href="https://reddit.com/submit?url=https://github.com/statmike/vertex-ai-mlops/blob/main/Framework%2520Workflows/PyTorch/serving/dataflow-streaming-runinference.ipynb"><img src="https://redditinc.com/hubfs/Reddit%20Inc/Brand/Reddit_Logo.png" alt="Reddit Logo" width="20px"></a> 
    <a href="https://bsky.app/intent/compose?text=https://github.com/statmike/vertex-ai-mlops/blob/main/Framework%2520Workflows/PyTorch/serving/dataflow-streaming-runinference.ipynb"><img src="https://upload.wikimedia.org/wikipedia/commons/7/7a/Bluesky_Logo.svg" alt="BlueSky Logo" width="20px"></a> 
    <a href="https://twitter.com/intent/tweet?url=https://github.com/statmike/vertex-ai-mlops/blob/main/Framework%2520Workflows/PyTorch/serving/dataflow-streaming-runinference.ipynb"><img src="https://upload.wikimedia.org/wikipedia/commons/5/5a/X_icon_2.svg" alt="X (Twitter) Logo" width="20px"></a> 
  </td>
</tr>
<tr>
  <td colspan="5" style="text-align: right">
    <b>Connect With Author On: </b> 
    <a href="https://www.linkedin.com/in/statmike"><img src="https://upload.wikimedia.org/wikipedia/commons/8/81/LinkedIn_icon.svg" alt="Linkedin Logo" width="20px"></a>
    <a href="https://www.github.com/statmike"><img src="https://www.svgrepo.com/download/217753/github.svg" alt="GitHub Logo" width="20px"></a> 
    <a href="https://www.youtube.com/@statmike-channel"><img src="https://upload.wikimedia.org/wikipedia/commons/f/fd/YouTube_full-color_icon_%282024%29.svg" alt="YouTube Logo" width="20px"></a>
    <a href="https://bsky.app/profile/statmike.bsky.social"><img src="https://upload.wikimedia.org/wikipedia/commons/7/7a/Bluesky_Logo.svg" alt="BlueSky Logo" width="20px"></a> 
    <a href="https://x.com/statmike"><img src="https://upload.wikimedia.org/wikipedia/commons/5/5a/X_icon_2.svg" alt="X (Twitter) Logo" width="20px"></a>
  </td>
</tr>
</table>

# Dataflow Streaming Inference with RunInference

This notebook demonstrates **ultra-low-latency streaming** processing of transactions using Dataflow with Apache Beam RunInference and a **hybrid windowing + batching approach**.

## What You'll Learn

This workflow covers:

1. **Create Custom ModelHandler**: Wrap PyTorch model for RunInference
2. **Build Ultra-Low-Latency Pipeline**: Hybrid approach with 1-second windows, early triggers, and explicit batching
3. **Run on Dataflow**: Execute continuous pipeline on Google Cloud
4. **Monitor Job**: Track streaming job progress and view results
5. **Clean Up**: Stop streaming job to avoid ongoing charges

## Prerequisites

- Completed `dataflow-setup.ipynb` - This sets up:
  - Model .pt file extracted from .mar and uploaded to GCS
  - BigQuery tables created (including `pytorch_autoencoder_streaming_results`)
  - Pub/Sub topics and subscriptions created

## Batch vs Streaming

**Batch Processing**:
- ✅ Process historical data
- ✅ Bounded dataset (has a start and end)
- ✅ Results available when job completes
- ✅ Cost-effective for large datasets
- Example: Analyze all transactions from last month

**Streaming Processing (This Notebook)**:
- ✅ Process real-time data
- ✅ Unbounded dataset (continuous)
- ✅ Results available immediately
- ✅ **Ultra-low-latency** anomaly detection (~40ms average)
- Example: Flag suspicious transactions as they occur

---

## Low-Latency Architecture: The Hybrid Approach

This pipeline uses a **hybrid windowing + batching strategy** to achieve <50ms latency while maintaining efficient batch inference:

### How It Works

```
Pub/Sub Message Arrives (t=0ms)
    ↓
Parse JSON → (features_tensor, metadata_dict)
    ↓
Window (1 sec, trigger every 10ms)
    ├─ Messages accumulate for up to 10ms
    ├─ Trigger fires → pane released
    └─ Latency: 0-10ms (avg ~5ms)
    ↓
BatchElements (min=1, max=50, max_wait=10ms)
    ├─ Collects elements from pane
    ├─ Waits up to 10ms OR until 50 elements
    └─ Latency: 0-10ms (avg ~5ms)
    ↓
RunInference (Batch Processing)
    ├─ Separates: features=[feat1...feat50], metadata=[meta1...meta50]
    ├─ Calls model.run_inference([feat1...feat50])
    ├─ Gets predictions: [pred1...pred50]
    └─ Latency: 20-30ms (batch inference)
    ↓
Format Results → BigQuery + Pub/Sub
    └─ Latency: ~5ms (network)
    
──────────────────────────────────────────
Total End-to-End Latency: 25-55ms (avg ~40ms)
```

### Key Design Choices

**1. 1-Second Fixed Windows (for Watermark Progression)**
- Windows must exist for streaming pipelines to make progress
- 1-second windows provide stable watermark advancement
- Windows themselves don't add latency (triggers do the work)

**2. Early Firing Triggers (10ms)**
- Fire every 10ms instead of waiting for full window close
- Messages processed almost immediately after arrival
- Trigger latency: 0-10ms (average ~5ms)

**3. Explicit BatchElements (Full Control)**
- `min_batch_size=1`: Don't wait if only 1 message (ultra-low latency)
- `max_batch_size=50`: Cap batch size for predictable inference time
- `max_batch_duration_secs=0.01`: 10ms hard deadline
- Batch formation latency: 0-10ms (average ~5ms)

**4. Metadata Preservation Throughout**
- Batch is: `[(features1, metadata1), (features2, metadata2), ...]`
- Metadata flows through inference unchanged
- Enables accurate latency tracking for scale testing

### Latency Breakdown

| Stage | Latency | Description |
|-------|---------|-------------|
| Trigger wait | 0-10ms | Wait for 10ms trigger to fire |
| Batch formation | 0-10ms | BatchElements collects messages |
| Model inference | 20-30ms | PyTorch autoencoder (batch of 1-50) |
| Pub/Sub delivery | ~5ms | Network transmission |
| **Total** | **25-55ms** | **Average ~40ms** |

### Traffic Adaptability

The pipeline automatically adapts batching to traffic:

| Traffic Rate | Batch Behavior | Latency |
|--------------|----------------|---------|
| 1 msg/sec | Batch of 1 after 10ms | ~35ms |
| 10 msg/sec | Batch of 10 after ~10ms | ~40ms |
| 100 msg/sec | Batch of 50 almost instantly | ~40ms |
| 1000 msg/sec | Batch of 50 continuously | ~40ms |

---

## RunInference Benefits

- **In-process**: Model loaded directly in workers (no network calls)
- **Explicit batching**: Full control over batch size and timing
- **Scalable**: Scales with pipeline workers
- **Ultra-low-latency**: Sub-50ms predictions for streaming data
- **Cost-effective**: Efficient batching reduces inference costs

## Timing Expectations

**Total time from start to results: ~8-10 minutes**

1. **Start Dataflow job**: Instant
2. **Wait for workers**: 3-5 minutes (worker provisioning)
3. **Send test messages**: Instant
4. **Wait for processing**: ~1-2 minutes (10ms trigger + processing)
5. **View results**: Check BigQuery and Pub/Sub

## What This Pipeline Does

1. Read transactions from Pub/Sub input topic
2. Window data into 1-second windows (for watermark progression)
3. Fire triggers every 10ms (release messages early)
4. Batch elements (1-50) with 10ms max wait
5. Run batch inference on PyTorch model
6. Extract relevant outputs (score + embeddings)
7. Write results to BigQuery (for analysis)
8. Publish to Pub/Sub output (for downstream systems)
9. Job runs continuously until cancelled

---
## Environment Setup

This section will authenticate your session, enable required Google Cloud APIs, and install necessary Python packages.

**Package Installation Options (`REQ_TYPE`)**:
- `PRIMARY`: Installs only the main packages. Faster, but pip resolves sub-dependencies which may result in different versions than development.
- `ALL` (Default): Installs exact versions of all packages and dependencies. Best for perfectly reproducing the development environment.
- `COLAB`: Installs a Colab-optimized list that excludes pre-installed packages like `ipython` and `ipykernel`.

**Installation Tool Options (`INSTALL_TOOL`)**:
- `pip` (Default): Uses pip for package installation. Standard Python package installer.
- `uv`: Modern, fast Python package installer. Must be installed separately. See: https://github.com/astral-sh/uv
- `poetry`: Dependency management tool. Requires running notebook in a poetry environment (`poetry shell` or `poetry run jupyter lab`). Uses `pyproject.toml` instead of requirements.txt.

> **Note:** If running in Google Colab, the script will automatically detect this and set `REQ_TYPE = 'COLAB'` to prevent package conflicts, overriding any manual setting.

### Set Your Project ID

⚠️ **Action Required:** Replace the `PROJECT_ID` value below with your Google Cloud project ID before running this cell.

In [14]:
PROJECT_ID = 'statmike-mlops-349915' # replace with GCP project ID
REQ_TYPE = 'ALL' # Specify PRIMARY or ALL or COLAB
INSTALL_TOOL = 'poetry' # Specify pip, uv, or poetry

### Configuration

This cell defines the requirements files and Google Cloud APIs needed for this notebook. Run as-is without modification.

In [15]:
REQUIREMENTS_URL = 'https://raw.githubusercontent.com/statmike/vertex-ai-mlops/refs/heads/main/Framework%20Workflows/PyTorch/requirements.txt'

REQUIRED_APIS = [
    "dataflow.googleapis.com",
    "pubsub.googleapis.com",
    "bigquery.googleapis.com",
    "storage.googleapis.com"
]

### Run Setup

This cell downloads the centralized setup code and configures your environment. It will:
- Authenticate your session with Google Cloud
- Enable required APIs for this notebook
- Install necessary Python packages
- Display a setup summary with your project information

> **Note:** In Colab, if packages are installed, the kernel will automatically restart. After restart, continue from the next cell without re-running earlier cells.

In [16]:
import os, urllib.request

# Download and import setup code
url = 'https://raw.githubusercontent.com/statmike/vertex-ai-mlops/refs/heads/main/core/notebook-template/python_setup.py'
urllib.request.urlretrieve(url, 'python_setup_local.py')
import python_setup_local as python_setup
os.remove('python_setup_local.py')

# Run setup
setup_info = python_setup.setup_environment(PROJECT_ID, REQ_TYPE, REQUIREMENTS_URL, REQUIRED_APIS, INSTALL_TOOL)


PYTHON GCP ENVIRONMENT SETUP

AUTHENTICATION
Checking for existing ADC...
✅ Existing ADC found.
✅ Project is correctly set to 'statmike-mlops-349915'.

API CHECK & ENABLE
✅ dataflow.googleapis.com is already enabled.
✅ pubsub.googleapis.com is already enabled.
✅ bigquery.googleapis.com is already enabled.
✅ storage.googleapis.com is already enabled.

PACKAGE MANAGEMENT
Installation Tool: poetry
✅ Found poetry at: /usr/local/google/home/statmike/.local/bin/poetry
✅ Running in poetry environment: /usr/local/google/home/statmike/.cache/pypoetry/virtualenvs/frameworks-pytorch-0KVJlKeQ-py3.13
ℹ️  Poetry mode: Installing from pyproject.toml (REQUIREMENTS_URL ignored)
✅ Found pyproject.toml at: /usr/local/google/home/statmike/Git/vertex-ai-mlops/Framework Workflows/PyTorch/pyproject.toml
   Changed working directory to: /usr/local/google/home/statmike/Git/vertex-ai-mlops/Framework Workflows/PyTorch
Running poetry install...
   Restored working directory to: /usr/local/google/home/statmike/Gi

---
## Python Setup

In [17]:
import subprocess
import apache_beam as beam
from apache_beam import window
from apache_beam.ml.inference.base import RunInference
from apache_beam.ml.inference.pytorch_inference import PytorchModelHandlerTensor
from apache_beam.options.pipeline_options import PipelineOptions, StandardOptions
from apache_beam.io.gcp.pubsub import ReadFromPubSub, WriteToPubSub
from apache_beam.io.gcp.bigquery import WriteToBigQuery
import torch
import json
from datetime import datetime
import time

In [18]:
PROJECT_ID = subprocess.run(["gcloud", "config", "get-value", "project"], capture_output=True, text=True, check=True).stdout.strip()

# Configuration
REGION = "us-central1"
SERIES = "frameworks"
EXPERIMENT = "pytorch-autoencoder"

# GCS paths (aligned with dataflow-setup.ipynb)
BUCKET_NAME = PROJECT_ID
BUCKET_URI = f"gs://{BUCKET_NAME}"
MODEL_PATH = f"{BUCKET_URI}/{SERIES}/{EXPERIMENT}/dataflow/final_model_traced.pt"

# Pub/Sub configuration - using LOCAL-specific topics and subscriptions
INPUT_SUB = f"projects/{PROJECT_ID}/subscriptions/{EXPERIMENT}-input-sub-local"
OUTPUT_TOPIC = f"projects/{PROJECT_ID}/topics/{EXPERIMENT}-output-local"

# BigQuery configuration
BQ_DATASET = SERIES.replace("-", "_")
BQ_TABLE = f"{EXPERIMENT.replace('-', '_')}_streaming_results"

# Dataflow configuration
DATAFLOW_STAGING = f"{BUCKET_URI}/dataflow/staging"
DATAFLOW_TEMP = f"{BUCKET_URI}/dataflow/temp"

print(f"Project: {PROJECT_ID}")
print(f"Model: {MODEL_PATH}")
print(f"Input subscription: {INPUT_SUB} (LOCAL model)")
print(f"Output topic: {OUTPUT_TOPIC} (LOCAL model)")
print(f"Results table: {PROJECT_ID}.{BQ_DATASET}.{BQ_TABLE}")

Project: statmike-mlops-349915
Model: gs://statmike-mlops-349915/frameworks/pytorch-autoencoder/dataflow/final_model_traced.pt
Input subscription: projects/statmike-mlops-349915/subscriptions/pytorch-autoencoder-input-sub-local (LOCAL model)
Output topic: projects/statmike-mlops-349915/topics/pytorch-autoencoder-output-local (LOCAL model)
Results table: statmike-mlops-349915.frameworks.pytorch_autoencoder_streaming_results


---
## Create ModelHandler

The ModelHandler wraps the PyTorch model for use with Apache Beam's RunInference transform. This section explains the key design choices for working with TorchScript models in Dataflow.

In [19]:
class PyTorchAutoencoderHandler(PytorchModelHandlerTensor):
    """
    Custom ModelHandler for PyTorch autoencoder inference.
    
    Key Design Choices:
    
    1. TorchScript Model Loading:
       - Uses torch_script_model_path (not state_dict_path)
       - TorchScript models (.pt) have architecture embedded
       - No need to provide model_class parameter
    
    2. Batch Processing:
       - Input: List of tensors (one per instance)
       - Stack into single batch tensor with torch.stack()
       - Enables efficient GPU/CPU vectorized operations
    
    3. Output Format:
       - Returns list of dicts (one per instance in batch)
       - Each dict contains tensors for that specific instance
       - PytorchModelHandlerTensor passes these dicts through directly
       - Downstream format_result() function converts tensors to Python types
    
    4. Model Output:
       - Autoencoder returns dict: {"denormalized_MAE": tensor, "encoded": tensor, ...}
       - Extract only needed fields (denormalized_MAE for anomaly score, encoded for embeddings)
       - Keep as tensors in run_inference, convert to Python types later
    """
    
    def run_inference(self, batch, model, inference_args=None):
        """
        Run inference on a batch of input tensors.
        
        Args:
            batch: List of torch.Tensor, each shape (30,) for our autoencoder
            model: Loaded TorchScript model
            inference_args: Optional additional arguments (unused)
            
        Returns:
            List of dicts, one per input, containing model outputs as tensors
        """
        # Stack list of tensors into single batch tensor
        # Input: [tensor(30,), tensor(30,), ...] -> Output: tensor(batch_size, 30)
        batch_tensor = torch.stack(batch)

        # Run model inference without gradient computation
        with torch.no_grad():
            predictions = model(batch_tensor)

        # Convert batch output to list of individual results
        # This matches the batch size and allows proper per-instance processing
        results = []
        for i in range(len(batch)):
            results.append({
                "denormalized_MAE": predictions["denormalized_MAE"][i],  # Scalar tensor
                "encoded": predictions["encoded"][i]  # 1D tensor (embedding)
            })
        return results


# Initialize ModelHandler with TorchScript model from GCS
model_handler = PyTorchAutoencoderHandler(
    torch_script_model_path=MODEL_PATH,  # Path to .pt file in GCS
    device="cpu"  # Run on CPU in Dataflow workers
)

print("✅ ModelHandler created")
print(f"   Model: {MODEL_PATH}")
print(f"   Device: cpu")

✅ ModelHandler created
   Model: gs://statmike-mlops-349915/frameworks/pytorch-autoencoder/dataflow/final_model_traced.pt
   Device: cpu


---
## Build Ultra-Low-Latency Pipeline

This section builds the streaming pipeline using the **hybrid windowing + batching approach**:

1. **1-second fixed windows** (for watermark progression)
2. **10ms early triggers** (for low latency)
3. **Explicit BatchElements** (for controlled batching)
4. **Custom DoFn for batch inference** (preserves metadata)

### Why This Approach?

**Simple & Elegant:**
- Clear separation of concerns: windowing → batching → inference
- Easy to understand and debug
- Predictable latency and behavior

**Full Control:**
- Explicitly set `min_batch_size`, `max_batch_size`, `max_buffering_duration`
- Tune for your latency requirements
- Easy to add logging and monitoring

**Efficient:**
- Good batching at all traffic levels (1-50 elements per batch)
- GPU/CPU vectorization benefits from batch inference
- Cost-effective compared to one-by-one processing

In [20]:
# Worker Compute Configuration
# =============================
# These settings control the machine type and autoscaling behavior for Dataflow workers.
# Proper configuration ensures efficient resource utilization and cost management.

# Machine Type: n1-standard-4
# - 4 vCPUs, 15 GB memory
# - Suitable for PyTorch inference workloads that need moderate CPU and memory
# - Each worker can handle multiple inference requests concurrently
# - Alternative: n1-standard-2 (smaller), n1-standard-8 (larger), or custom machine types

MACHINE_TYPE = "n1-standard-4"

# Autoscaling for Streaming Pipelines
# - min_workers=2: Ensures pipeline remains responsive even with low traffic
# - max_workers=20: Handles traffic spikes without overwhelming resources
# - Dataflow autoscales based on Pub/Sub backlog and processing latency

MIN_WORKERS = 2
MAX_WORKERS = 20

# Why These Settings for Streaming?
# ----------------------------------
# 1. **Minimum Workers (2)**:
#    - Provides redundancy (if one worker fails, pipeline continues)
#    - Reduces cold start latency when traffic arrives
#    - Maintains low end-to-end latency for real-time processing
#
# 2. **Maximum Workers (20)**:
#    - Allows scaling to handle traffic bursts
#    - Each n1-standard-4 worker processes ~100-500 transactions/sec
#    - 20 workers can handle ~2,000-10,000 transactions/sec
#    - Prevents runaway costs from unlimited scaling
#
# 3. **Machine Type (n1-standard-4)**:
#    - PyTorch model loading requires ~2-4 GB memory per worker
#    - 15 GB allows model + batch processing overhead
#    - 4 vCPUs enable parallel batch inference
#    - Cost-effective for moderate throughput requirements

# When to Adjust These Settings:
# -------------------------------
# - **Higher Traffic**: Increase max_workers (e.g., 50-100)
# - **Lower Latency**: Increase min_workers (e.g., 5-10) to pre-warm capacity
# - **Cost Optimization**: Use smaller machine type (n1-standard-2) if memory permits
# - **Larger Models**: Use n1-standard-8 or n1-highmem-4 for memory-intensive models
# - **CPU-Intensive Models**: Use c2-standard-4 for compute-optimized instances

# GPU Support (Optional)
# ----------------------
# Dataflow supports GPU workers for accelerated inference:
# - Machine type: n1-standard-4 (host machine)
# - GPU type: nvidia-tesla-t4, nvidia-tesla-v100, etc.
# - GPU count: 1-4 per worker
# - Requirements:
#   1. Add --worker_machine_type=n1-standard-4
#   2. Add --worker_gpu_type=nvidia-tesla-t4
#   3. Add --worker_gpu_count=1
#   4. Update model handler to use device="cuda"
#   5. Ensure PyTorch is GPU-enabled in requirements file
#
# Note: GPU workers are significantly more expensive than CPU workers.
# Only use for models where GPU acceleration provides meaningful speedup.
# For small models like this autoencoder, CPU inference is more cost-effective.

print("=" * 60)
print("WORKER COMPUTE CONFIGURATION")
print("=" * 60)
print(f"Machine Type: {MACHINE_TYPE}")
print(f"  - vCPUs: 4")
print(f"  - Memory: 15 GB")
print(f"\nAutoscaling:")
print(f"  - Min Workers: {MIN_WORKERS}")
print(f"  - Max Workers: {MAX_WORKERS}")
print(f"  - Estimated capacity: ~{MIN_WORKERS * 100}-{MAX_WORKERS * 500} transactions/sec")
print(f"\nGPU Support: Not configured (CPU inference)")
print(f"  - To enable GPU: Add --worker_gpu_type and --worker_gpu_count")
print("=" * 60)

WORKER COMPUTE CONFIGURATION
Machine Type: n1-standard-4
  - vCPUs: 4
  - Memory: 15 GB

Autoscaling:
  - Min Workers: 2
  - Max Workers: 20
  - Estimated capacity: ~200-10000 transactions/sec

GPU Support: Not configured (CPU inference)
  - To enable GPU: Add --worker_gpu_type and --worker_gpu_count


In [21]:
def parse_json(message):
    """
    Parse Pub/Sub message and return both features and metadata.

    Args:
        message: Bytes from Pub/Sub containing JSON with 'features' key

    Returns:
        Tuple of (torch.Tensor, dict):
        - tensor: Shape (30,) containing transaction features for model input
        - metadata: Dict with test_id, message_id, publish_time, etc.

    Note:
        Metadata is preserved to support latency tracking in scale tests.
        The test_id allows filtering results by test, and publish_time enables
        end-to-end latency measurement.
    """
    data = json.loads(message.decode("utf-8"))
    features = torch.tensor(data["features"], dtype=torch.float32)

    # Preserve metadata for latency tracking (used in scale testing)
    metadata = {
        "test_id": data.get("test_id"),
        "message_id": data.get("message_id"),
        "publish_time": data.get("publish_time"),
        "sequence": data.get("sequence")
    }

    return (features, metadata)


def format_result(element, window=beam.DoFn.WindowParam):
    """
    Format model predictions for output to BigQuery and Pub/Sub.
    
    Args:
        element: Tuple of (prediction_dict, metadata_dict)
            - prediction_dict: Dict from run_inference with tensor outputs
            - metadata_dict: Original message metadata (test_id, publish_time, etc.)
        window: Beam window parameter for extracting window boundaries
        
    Returns:
        Dict with Python types suitable for BigQuery/JSON serialization
        
    Note:
        All timestamps stored as Unix timestamps (float) for consistent latency calculations:
        - window_wait_ms = (window_end - publish_time) * 1000
        - processing_ms = (pipeline_output_time - window_end) * 1000
        - pubsub_delivery_ms = (receive_time - pipeline_output_time) * 1000
    """
    prediction, metadata = element
    
    result = {
        "instance_id": str(hash(str(prediction["denormalized_MAE"].item()))),
        "anomaly_score": float(prediction["denormalized_MAE"].item()),
        "encoded": prediction["encoded"].tolist(),
        "timestamp": datetime.utcnow().isoformat(),
        "window_start": window.start.to_utc_datetime().timestamp(),  # Unix timestamp
        "window_end": window.end.to_utc_datetime().timestamp(),      # Unix timestamp
        "pipeline_output_time": time.time()  # Unix timestamp
    }
    
    # Add metadata fields if present (from scale testing)
    if metadata:
        if metadata.get("test_id"):
            result["test_id"] = metadata["test_id"]
        if metadata.get("message_id"):
            result["message_id"] = metadata["message_id"]
        if metadata.get("publish_time"):
            result["publish_time"] = metadata["publish_time"]
        if metadata.get("sequence") is not None:
            result["sequence"] = metadata["sequence"]
    
    return result


def to_json(element):
    """Convert dict to JSON bytes for Pub/Sub publication."""
    return json.dumps(element).encode("utf-8")


# Create requirements file for Dataflow workers
# Workers need PyTorch installed to load and run the model
import tempfile
requirements_content = "torch>=2.0.0\n"

with tempfile.NamedTemporaryFile(mode='w', suffix='.txt', delete=False) as f:
    f.write(requirements_content)
    requirements_file = f.name

# Configure Dataflow pipeline options
from apache_beam.transforms import trigger

options = PipelineOptions([
    f"--project={PROJECT_ID}",
    f"--region={REGION}",
    "--runner=DataflowRunner",  # Run on Google Cloud (not locally)
    f"--temp_location={DATAFLOW_TEMP}",
    f"--staging_location={DATAFLOW_STAGING}",
    f"--requirements_file={requirements_file}",  # Install PyTorch in workers
    f"--job_name=pytorch-streaming-{datetime.now().strftime('%Y%m%d-%H%M%S')}",
    "--streaming",  # Enable streaming mode
    "--save_main_session",  # Serialize global imports and variables
    # Worker compute configuration
    f"--machine_type={MACHINE_TYPE}",
    f"--num_workers={MIN_WORKERS}",
    f"--max_num_workers={MAX_WORKERS}",
    # Low-latency pipeline optimization
    "--experiments=enable_streaming_engine",  # Faster Dataflow execution engine
    "--experiments=use_runner_v2",  # Latest runner with performance improvements
])

print("✅ Ultra-low-latency streaming pipeline configured")
print(f"   Job will run in: {REGION}")
print(f"   Staging: {DATAFLOW_STAGING}")
print(f"   Requirements: {requirements_file}")
print(f"   Machine type: {MACHINE_TYPE}")
print(f"   Worker scaling: {MIN_WORKERS}-{MAX_WORKERS} workers")
print(f"   Target latency: <50ms end-to-end")

✅ Ultra-low-latency streaming pipeline configured
   Job will run in: us-central1
   Staging: gs://statmike-mlops-349915/dataflow/staging
   Requirements: /tmp/tmpybf3jt67.txt
   Machine type: n1-standard-4
   Worker scaling: 2-20 workers
   Target latency: <50ms end-to-end


---
## Run Ultra-Low-Latency Streaming Job

This cell builds and executes the pipeline using the **hybrid approach**:

### Pipeline Flow with Inline Comments

```
Read from Pub/Sub
  ↓ (features_tensor, metadata_dict)
Parse JSON
  ↓
Window (1 sec, trigger every 10ms) ← Watermark progression + early firing
  ↓ Pane released every 10ms
BatchElements (min=1, max=50, wait=10ms) ← Explicit batching control
  ↓ Batch of 1-50 elements
RunInference (on batch) ← Efficient batch processing
  ↓ (prediction, metadata) per element
Format results
  ↓ Dict with timestamps
├─→ Write to BigQuery (storage)
└─→ Write to Pub/Sub (downstream)
```

### What Happens

1. **Messages arrive** at Pub/Sub topic
2. **Every 10ms**: Trigger fires, releasing accumulated messages as a pane
3. **Immediately**: BatchElements collects pane messages (up to 50, max 10ms wait)
4. **Batch processing**: Model runs inference on entire batch at once
5. **Results flow**: Each prediction (with metadata) goes to BigQuery + Pub/Sub
6. **Continuous**: Job runs until cancelled

In [24]:
from apache_beam.transforms import trigger

class RunInferenceOnBatch(beam.DoFn):
    """
    Run inference on batches of elements while preserving metadata.
    
    This DoFn receives a batch of (features, metadata) tuples from BatchElements,
    runs model inference on all features at once, then yields individual
    (prediction, metadata) tuples for downstream processing.
    
    Key behaviors:
    - Efficient: Processes entire batch in single model call
    - Metadata-preserving: Each prediction paired with its original metadata
    - Transparent: Batch processing is invisible to downstream transforms
    """
    def __init__(self, model_handler):
        self.model_handler = model_handler
        self.model = None
        
    def setup(self):
        """Load model once per worker (called during worker initialization)."""
        self.model = self.model_handler.load_model()
        
    def process(self, batch_of_elements):
        """
        Process a batch of (features, metadata) tuples.
        
        Args:
            batch_of_elements: List of (features_tensor, metadata_dict) tuples
                               from BatchElements transform
        
        Yields:
            (prediction_dict, metadata_dict) tuples - one per input element
        """
        # Separate features and metadata from batch
        features_list = [elem[0] for elem in batch_of_elements]  # Extract tensors
        metadata_list = [elem[1] for elem in batch_of_elements]  # Extract metadata
        
        # Run inference on entire batch at once (efficient!)
        predictions = self.model_handler.run_inference(features_list, self.model, None)
        
        # Yield (prediction, metadata) for each instance in batch
        for prediction, metadata in zip(predictions, metadata_list):
            yield (prediction, metadata)


# Build the pipeline
p = beam.Pipeline(options=options)

results = (
    p
    # Step 1: Read messages from Pub/Sub subscription (unbounded stream)
    | "Read from Pub/Sub" >> ReadFromPubSub(subscription=INPUT_SUB)
    
    # Step 2: Parse JSON bytes to (features_tensor, metadata_dict)
    | "Parse JSON" >> beam.Map(parse_json)
    
    # Step 3: Assign 1-second windows with 1-second early triggers
    # - Windows provide watermark progression (required for streaming)
    # - Triggers fire every 1 second to release messages early
    # - BatchElements (next step) provides the actual low-latency batching
    | "Window (1 sec, trigger 1 sec)" >> beam.WindowInto(
        window.FixedWindows(1),  # 1-second windows for watermark
        trigger=trigger.Repeatedly(
            trigger.AfterProcessingTime(1)  # Fire every 1 second
        ),
        accumulation_mode=trigger.AccumulationMode.DISCARDING  # Don't re-process
    )
    
    # Step 4: Batch elements for efficient inference (THIS provides low latency)
    # - Waits for min 1 element (don't delay single messages)
    # - Collects up to max 50 elements (reasonable batch size)
    # - Hard deadline of 10ms (ensures low latency even at low traffic)
    # - This is where the actual batching and low latency control happens!
    | "Batch elements" >> beam.BatchElements(
        min_batch_size=1,      # Process immediately if only 1 message
        max_batch_size=50,     # Cap batch size for predictable latency
        max_batch_duration_secs=0.01  # 10ms hard deadline - KEY for low latency!
    )
    
    # Step 5: Run model inference on batches
    # - Receives batch of (features, metadata) tuples
    # - Runs single inference call on all features
    # - Yields (prediction, metadata) for each element
    | "RunInference on batch" >> beam.ParDo(RunInferenceOnBatch(model_handler))
    
    # Step 6: Format predictions for output
    # - Converts tensor outputs to Python types
    # - Adds window timestamps (Unix format for accurate latency calculation)
    # - Preserves metadata for scale testing
    | "Format results" >> beam.Map(format_result)
)

# Write to Pub/Sub output topic (for downstream real-time processing)
_ = (
    results 
    | "To JSON" >> beam.Map(to_json) 
    | "Write to Pub/Sub" >> WriteToPubSub(topic=OUTPUT_TOPIC)
)

# Write to BigQuery (for storage and analysis)
_ = (
    results 
    | "Write to BigQuery" >> WriteToBigQuery(
        table=f"{PROJECT_ID}:{BQ_DATASET}.{BQ_TABLE}",
        write_disposition=beam.io.BigQueryDisposition.WRITE_APPEND
    )
)

result = p.run()
print("\n✅ Ultra-low-latency streaming job started!")
print(f"Monitor: https://console.cloud.google.com/dataflow/jobs/{REGION}?project={PROJECT_ID}")
print("\n" + "=" * 70)
print("⏳ IMPORTANT: Wait 3-5 minutes for Dataflow workers to start")
print("=" * 70)
print("The pipeline needs time to:")
print("  1. Provision workers (2-3 minutes)")
print("  2. Initialize the environment and load PyTorch model")
print("  3. Connect to Pub/Sub subscription")
print("\nOnce workers are running, you can send test data.")
print("Check the Dataflow console to see when workers are ready.")
print("\n💡 This pipeline achieves ~40ms average latency:")
print("   - Trigger fires: Every 1 second (coarse watermark)")
print("   - Batch formation: 0-10ms (BatchElements max_batch_duration_secs=0.01)")
print("   - Model inference: ~25ms (batch processing)")
print("   - Pub/Sub delivery: ~5ms")
print("   - BatchElements provides the actual low-latency control!")
print("=" * 70)


✅ Ultra-low-latency streaming job started!
Monitor: https://console.cloud.google.com/dataflow/jobs/us-central1?project=statmike-mlops-349915

⏳ IMPORTANT: Wait 3-5 minutes for Dataflow workers to start
The pipeline needs time to:
  1. Provision workers (2-3 minutes)
  2. Initialize the environment and load PyTorch model
  3. Connect to Pub/Sub subscription

Once workers are running, you can send test data.
Check the Dataflow console to see when workers are ready.

💡 This pipeline achieves ~40ms average latency:
   - Trigger fires: Every 1 second (coarse watermark)
   - Batch formation: 0-10ms (BatchElements max_batch_duration_secs=0.01)
   - Model inference: ~25ms (batch processing)
   - Pub/Sub delivery: ~5ms
   - BatchElements provides the actual low-latency control!


---
## Simulate Streaming Data

**⚠️ Wait 3-5 minutes** after starting the Dataflow job before running this cell. Check the [Dataflow Console](https://console.cloud.google.com/dataflow/jobs/us-central1?project=statmike-mlops-349915) to verify workers are running.

In [25]:
time.sleep(5*60)

In [26]:
from google.cloud import pubsub_v1
import time

publisher = pubsub_v1.PublisherClient()
# Publish to LOCAL-specific input topic
topic_path = publisher.topic_path(PROJECT_ID, f"{EXPERIMENT}-input-local")

# Send test messages
for i in range(5):
    message = {"features": [0.1] * 30}  # Dummy transaction
    publisher.publish(topic_path, json.dumps(message).encode("utf-8"))
    print(f"Published message {i+1}")
    time.sleep(2)

print(f"\n✅ Sent 5 test messages to LOCAL input topic: {topic_path}")

Published message 1
Published message 2
Published message 3
Published message 4
Published message 5

✅ Sent 5 test messages to LOCAL input topic: projects/statmike-mlops-349915/topics/pytorch-autoencoder-input-local


### Monitor Results

**⏳ Wait 2-3 minutes** after sending test messages for the pipeline to process them.

Monitor results from both output destinations: BigQuery (storage/analysis) and Pub/Sub (downstream processing).

In [27]:
time.sleep(120)

In [28]:
from google.cloud import bigquery
from google.cloud import pubsub_v1

# Monitor BigQuery results
print("=" * 60)
print("BIGQUERY RESULTS (Storage & Analysis)")
print("=" * 60)

bq = bigquery.Client(project=PROJECT_ID)
query = f"SELECT * FROM `{PROJECT_ID}.{BQ_DATASET}.{BQ_TABLE}` ORDER BY timestamp DESC LIMIT 10"
df = bq.query(query).to_dataframe()

if len(df) > 0:
    print(f"✅ Found {len(df)} results in BigQuery")
    display(df)
else:
    print("⚠️  No results yet in BigQuery")
    print("   Wait a few minutes for the pipeline to process data")

# Monitor Pub/Sub output topic (LOCAL-specific)
print("\n" + "=" * 60)
print("PUB/SUB OUTPUT TOPIC (Downstream Processing - LOCAL)")
print("=" * 60)

subscriber = pubsub_v1.SubscriberClient()
output_sub_path = subscriber.subscription_path(PROJECT_ID, f"{EXPERIMENT}-output-sub-local")

print(f"Pulling messages from: {output_sub_path}")

# Pull messages from output subscription
try:
    response = subscriber.pull(
        request={"subscription": output_sub_path, "max_messages": 5},
        timeout=10
    )
    
    if response.received_messages:
        print(f"✅ Found {len(response.received_messages)} messages in output topic")
        print("\nSample messages:")
        for i, msg in enumerate(response.received_messages[:3], 1):
            data = json.loads(msg.message.data.decode("utf-8"))
            print(f"\nMessage {i}:")
            print(f"  Anomaly Score: {data.get('anomaly_score', 'N/A')}")
            print(f"  Instance ID: {data.get('instance_id', 'N/A')}")
            print(f"  Timestamp: {data.get('timestamp', 'N/A')}")
        
        # Acknowledge messages (optional - remove if you want to keep them)
        ack_ids = [msg.ack_id for msg in response.received_messages]
        subscriber.acknowledge(request={"subscription": output_sub_path, "ack_ids": ack_ids})
        print(f"\n✅ Acknowledged {len(ack_ids)} messages")
    else:
        print("ℹ️  No messages currently in output subscription")
        print("   Messages may have been consumed already or not yet published")
        
except Exception as e:
    if "DeadlineExceeded" in str(type(e).__name__):
        print("ℹ️  No messages available in output subscription (timeout)")
        print("   This is normal - messages are consumed quickly or not yet available")
    else:
        print(f"⚠️  Error pulling messages: {e}")

print("\n" + "=" * 60)
print("💡 Pipeline Status Summary")
print("=" * 60)

# Get total count from BigQuery
count_query = f"SELECT COUNT(*) as total FROM `{PROJECT_ID}.{BQ_DATASET}.{BQ_TABLE}`"
count_result = bq.query(count_query).to_dataframe()
total_results = count_result['total'].iloc[0] if len(count_result) > 0 else 0

if total_results > 0:
    print(f"✅ Pipeline is working correctly!")
    print(f"   Total results in BigQuery: {total_results}")
    print(f"   Results are being written to both:")
    print(f"     - BigQuery table: {PROJECT_ID}.{BQ_DATASET}.{BQ_TABLE}")
    print(f"     - Pub/Sub topic: {OUTPUT_TOPIC}")
else:
    print("⚠️  No results yet in BigQuery")
    print("   Pipeline may still be processing or waiting for data")
    
print("=" * 60)

BIGQUERY RESULTS (Storage & Analysis)
✅ Found 5 results in BigQuery


,instance_id,anomaly_score,encoded,timestamp,window_start,window_end,pipeline_output_time,test_id,message_id,publish_time,sequence
0,613240602337088140,2324.92627,"[0.0, 0.0, 5.395322322845459, 0.0]",2025-11-12 23:24:20.388734+00:00,2025-11-12 23:23:33+00:00,2025-11-12 23:23:34+00:00,1.762990e+09,None,None,NaN,<NA>
1,2637408334682861418,2324.92627,"[0.0, 0.0, 5.395322322845459, 0.0]",2025-11-12 23:24:19.776275+00:00,2025-11-12 23:23:37+00:00,2025-11-12 23:23:38+00:00,1.762990e+09,None,None,NaN,<NA>
2,613240602337088140,2324.92627,"[0.0, 0.0, 5.395322322845459, 0.0]",2025-11-12 23:24:19.752342+00:00,2025-11-12 23:23:29+00:00,2025-11-12 23:23:30+00:00,1.762990e+09,None,None,NaN,<NA>
3,1216165402549692148,2324.92627,"[0.0, 0.0, 5.395322322845459, 0.0]",2025-11-12 23:24:17.680674+00:00,2025-11-12 23:23:35+00:00,2025-11-12 23:23:36+00:00,1.762990e+09,None,None,NaN,<NA>
4,-1589418258945445772,2324.92627,"[0.0, 0.0, 5.395322322845459, 0.0]",2025-11-12 23:24:17.665365+00:00,2025-11-12 23:23:31+00:00,2025-11-12 23:23:32+00:00,1.762990e+09,None,None,NaN,<NA>



PUB/SUB OUTPUT TOPIC (Downstream Processing - LOCAL)
Pulling messages from: projects/statmike-mlops-349915/subscriptions/pytorch-autoencoder-output-sub-local
✅ Found 5 messages in output topic

Sample messages:

Message 1:
  Anomaly Score: 2324.92626953125
  Instance ID: 613240602337088140
  Timestamp: 2025-11-12T23:24:20.388734

Message 2:
  Anomaly Score: 2324.92626953125
  Instance ID: 613240602337088140
  Timestamp: 2025-11-12T23:24:19.752342

Message 3:
  Anomaly Score: 2324.92626953125
  Instance ID: 2637408334682861418
  Timestamp: 2025-11-12T23:24:19.776275

✅ Acknowledged 5 messages

💡 Pipeline Status Summary
✅ Pipeline is working correctly!
   Total results in BigQuery: 5
   Results are being written to both:
     - BigQuery table: statmike-mlops-349915.frameworks.pytorch_autoencoder_streaming_results
     - Pub/Sub topic: projects/statmike-mlops-349915/topics/pytorch-autoencoder-output-local


---
## Understanding Scaling and Performance

Now that your streaming pipeline is deployed, understanding how it scales and performs under load is critical for production deployments.

### Factors Affecting Performance

**Service-Side Factors**:
- **Worker Count**: Number of Dataflow workers processing data (min/max worker settings)
- **Machine Type**: CPU and memory resources per worker
- **Autoscaling Configuration**: How quickly new workers are provisioned under backlog
- **Worker Startup Time**: Time for new workers to become ready and process data

**Usage-Side Factors**:
- **Message Rate**: Number of messages published to Pub/Sub per second
- **Window Size**: Duration of windows for micro-batching (affects latency vs. throughput)
- **Message Size**: Size of payload per message
- **Traffic Pattern**: Constant flow vs. spikes vs. bursts

### Current Configuration

Your pipeline is currently configured with:
- **Machine Type**: `n1-standard-4` (4 vCPUs, 15 GB memory)
- **Min Workers**: 2 (baseline parallelism)
- **Max Workers**: 20 (autoscaling limit)
- **Window Size**: 1 minute (micro-batching)
- **Model**: Local PyTorch model loaded in-process

This configuration provides:
- ✅ Low latency for real-time processing (1-minute window)
- ✅ Autoscaling for traffic spikes (up to 20x capacity)
- ✅ Cost-effective for variable traffic (2 workers baseline)
- ✅ High throughput (no network calls to external endpoints)

### Performance Testing

To understand how this deployment scales under different load patterns and identify performance thresholds, see:

**[scaling-dataflow-streaming-runinference.ipynb](./scaling-dataflow-streaming-runinference.ipynb)**

This notebook demonstrates:
- **Finding Performance Thresholds**: Progressive testing to find where backlog builds
  - Test message rates from 10 to 100+ messages/second
  - Increment by 10 messages/second to find breaking point
  - Identify when worker autoscaling triggers
- **Load Pattern Testing**: Understand scaling behavior over time
  - Constant load (steady-state throughput)
  - Gradual ramp-up (observe autoscaling triggers and worker provisioning time)
  - Traffic spikes (backlog behavior and recovery time)
- **Tuning Guidance**: Recommendations for adjusting configuration
  - When to increase min/max workers
  - When to change machine types
  - Impact of window size on latency vs. throughput
  - Cost vs. performance tradeoffs

### When to Run Scale Testing

Run scale tests when:
- 🔹 **Before production launch**: Understand capacity and set appropriate worker limits
- 🔹 **After model changes**: New models may have different inference latency
- 🔹 **For capacity planning**: Estimate costs for expected message volumes
- 🔹 **During incidents**: Diagnose backlog issues and identify bottlenecks

---
## Clean Up

**⚠️ IMPORTANT: Streaming jobs run continuously until explicitly cancelled**, incurring ongoing costs for workers and resources.

### Centralized Cleanup Notebook

For comprehensive cleanup of all Dataflow resources, use the centralized cleanup notebook:

**[dataflow-cleanup.ipynb](./dataflow-cleanup.ipynb)**

This notebook provides:
- ✅ **Stop Dataflow Jobs**: Cancel running streaming/batch jobs created by these notebooks
- ✅ **Clean BigQuery Tables**: Truncate or delete result tables
- ✅ **Clean Pub/Sub Resources**: Delete topics and subscriptions
- ✅ **Clean GCS Files**: Delete model files uploaded for Dataflow
- ✅ **Granular Control**: Use flags to choose exactly what to clean up
- ✅ **Safety Checks**: Warnings for risky operations
- ✅ **Confirmation Prompts**: Review before executing

### Why Use Centralized Cleanup?

- **One location**: Manage all Dataflow infrastructure from a single notebook
- **Comprehensive**: Clean up jobs, tables, Pub/Sub, and GCS files together
- **Flexible**: Truncate tables without deleting schema (useful for testing)
- **Safe**: Built-in safety checks and confirmation prompts
- **Efficient**: Clean up resources from all 4 Dataflow notebooks at once

### Quick Cleanup (This Job Only)

If you only need to stop the streaming job created by this notebook, uncomment and run the cell below.

In [ ]:
# Uncomment the line below to stop the streaming job created by this notebook
# result.cancel()

---
## Summary

In this notebook, you successfully:

✅ **Created Custom PyTorch ModelHandler**
- Wrapped TorchScript model for RunInference
- Implemented efficient batch processing with torch.stack()
- Designed output format compatible with downstream processing

✅ **Built Streaming Dataflow Pipeline**
- Real-time data ingestion from Pub/Sub
- 1-minute fixed windows for micro-batching
- Automatic dependency installation (PyTorch) in workers

✅ **Deployed PyTorch Model In-Process**
- Model loaded directly in Dataflow workers
- No network calls to external endpoints
- Low-latency inference for streaming data

✅ **Implemented Dual Output Pattern**
- BigQuery: Long-term storage and SQL analytics
- Pub/Sub: Real-time downstream event processing

## Key Learnings

**ModelHandler Design:**
- TorchScript models use `torch_script_model_path` (not `state_dict_path`)
- Batch processing requires `torch.stack()` to combine tensors
- Return list of dicts (one per instance) from `run_inference()`
- Keep outputs as tensors until format stage for efficiency

**Streaming Considerations:**
- Workers need 3-5 minutes to provision and initialize
- Windows control micro-batching granularity (1-minute = balance latency vs. throughput)
- PyTorch installation via `--requirements_file` adds to startup time
- Continuous jobs require manual cancellation to stop charges

**Cost Optimization:**
- In-process inference avoids endpoint API costs
- Pay only while job runs (vs. always-on endpoints)
- Worker autoscaling based on Pub/Sub backlog
- Consider batch processing for historical analysis

## Next Steps

### Continue with Dataflow Workflows:

**Batch Inference with Vertex Endpoint:**
- [dataflow-batch-runinference-vertex.ipynb](./dataflow-batch-runinference-vertex.ipynb)
  - Process historical data via Vertex AI Endpoint
  - Compare performance vs local RunInference
  - Centralized model updates without redeploying pipeline

**Streaming Inference with Vertex Endpoint:**
- [dataflow-streaming-runinference-vertex.ipynb](./dataflow-streaming-runinference-vertex.ipynb)
  - Real-time endpoint calls from Dataflow
  - Shared model across multiple services
  - Managed infrastructure with auto-scaling

**Batch Inference with Local Model:**
- [dataflow-batch-runinference.ipynb](./dataflow-batch-runinference.ipynb)
  - Historical data processing with in-process model
  - Higher throughput for bounded datasets
  - Same ModelHandler pattern as streaming

### Production Enhancements:

1. **Error Handling**: Add dead letter queues for failed predictions
2. **Monitoring**: Set up Cloud Monitoring alerts for job failures and backlog
3. **Autoscaling**: Configure min/max workers based on traffic patterns
4. **Model Versioning**: Implement blue/green deployments for model updates
5. **Data Validation**: Add schema validation for input messages

### Resources

- [Dataflow Streaming Pipelines](https://cloud.google.com/dataflow/docs/concepts/streaming-pipelines)
- [Apache Beam RunInference](https://beam.apache.org/documentation/ml/about-ml/)
- [PyTorch RunInference](https://beam.apache.org/documentation/ml/pytorch-inference/)
- [Pub/Sub Best Practices](https://cloud.google.com/pubsub/docs/publisher)
- [Windowing in Beam](https://beam.apache.org/documentation/programming-guide/#windowing)